In [1]:
import warnings
warnings.filterwarnings("ignore")

import re
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch

from DataLoader.DataLoader import DataLoader
from DataLoader.DataBasedAgent import DataBasedAgent
from DataLoader.DataRLAgent import DataRLAgent
import DeepRLAgent.VanillaInput.Train as Train
from PatternDetectionInCandleStick.Evaluation import Evaluation
import distinctipy

from importlib import reload

Train = reload(Train)
DeepRL = Train.Train
from utils_best_arm import add_train_portfo, add_test_portfo, plot_return, calc_return, plot_action_point, setup_logger
pd.options.display.max_colwidth = 100

device = "cpu"
CURRENT_PATH = os.getcwd()

In [2]:
def train(
    DATASET_NAME, 
    split_point='2018-01-01', 
    begin_date='2010-01-01', 
    end_date='2020-08-24', 
    initial_investment=1000,
    transaction_cost=0.0001,
    load_from_file=True,
    reward_type="profit",
    seed=42, 
    state_mode=1,
    n_episodes=5,
    lamb=0.0001,
    GAMMA=0.7, 
    n_step=5, 
    BATCH_SIZE=10, 
    ReplayMemorySize=20,
    TARGET_UPDATE=5,
    window_size=None, 
    train_portfolios={},
    test_portfolios={},
    arms=[],
    show_all = False,
    ratio_threshold=0.9,
):
    data_loader = DataLoader(DATASET_NAME, split_point=split_point, begin_date=begin_date, end_date=end_date, load_from_file=load_from_file)
    
    dataTrain_agent = DataRLAgent(data_loader.data_train, state_mode, 'action_encoder_decoder', device, GAMMA, n_step, BATCH_SIZE, window_size, transaction_cost)
    dataTest_agent = DataRLAgent(data_loader.data_test, state_mode, 'action_encoder_decoder', device, GAMMA, n_step, BATCH_SIZE, window_size, transaction_cost)
    
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)

    agent = DeepRL(data_loader, dataTrain_agent, dataTest_agent, 
                DATASET_NAME,  state_mode, window_size, transaction_cost,
                BATCH_SIZE=BATCH_SIZE, GAMMA=GAMMA, ReplayMemorySize=ReplayMemorySize,
                TARGET_UPDATE=TARGET_UPDATE, n_step=n_step, arms=arms)
    
    path = f"./Results/{DATASET_NAME}/{begin_date}~{end_date}/{seed}/train"
    arm = arms[0]
    name = f'{arm["name"]}_{arm["lamb"]}'
    model_path = f"{path}/model_{name}_{seed}.pkl"
    agent_test = agent.test(initial_investment=initial_investment, test_type='test', model_path=model_path)
    test_portfolio = agent_test.get_daily_portfolio_value()
    test_portfolio = pd.Series(test_portfolio).pct_change(1).fillna(0).values.tolist() 
    model_name = f'DQN-stock:{DATASET_NAME}-reward:{name}-seed:{seed}'
    return data_loader, {"name": model_name, "portfo": test_portfolio}

In [3]:
initial_investment = 1000


kwargs = {
    "load_from_file": True, 
    "transaction_cost": 0.0000,
    "initial_investment": initial_investment,
    "state_mode": 1,
    "GAMMA": 0.7, 
    "n_step": 5, 
    "BATCH_SIZE": 10, 
    "ReplayMemorySize": 20,
    "TARGET_UPDATE": 5,
    "window_size": None, 
    "lamb": 0.0,
}

_file = "AAPL"

_begin_date = '20{}-01-01'
_end_date = '20{}-01-01'
_split_point = '20{}-01-01' 

arms = [
    { "name": "profit", "lamb": 0},
    { "name": "regularized", "lamb": 0.01},
    { "name": "regularized", "lamb": 0.05},
    { "name": "regularized", "lamb": 0.1},
    { "name": "regularized", "lamb": 0.2},
    { "name": "sharpe", "lamb": 0.01 },
    { "name": "volatility", "lamb": 10 },
]

results = []
files = sorted(os.listdir("./Data/"))
for _file in files[:1]:
    print(_file)
    for idx, arm in enumerate(arms):
        results2 = []
        for seed in tqdm(range(10)):
            
            ls = []
            bhs = []
        
            train_portfolios = {}
            test_portfolios = {}
            
            for year in range(4):
                begin_date = _begin_date.format(16+year)
                end_date = _end_date.format(19+year)
                split_point = _split_point.format(18+year)

                kwargs.update({
                    "begin_date": begin_date, 
                    "end_date": end_date, 
                    "split_point": split_point, 
                    "DATASET_NAME": _file,
                    "reward_type": "",
                    "seed": seed,
                    "n_episodes": 140,
                    "arms": [arm],
                    "show_all": True,
                    "ratio_threshold": 3,
                    "train_portfolios": train_portfolios,
                    "test_portfolios": test_portfolios,
                })

                data_loader, model = train(**kwargs)
                ls.extend(model["portfo"])
                bh = data_loader.data_test_with_date["close"]
                bhs.append(bh)
            
            add_test_portfo(test_portfolios, seed, ls)
            if seed == 0: 
                bhs = pd.concat(bhs, axis=0)
                bh_percentage = bhs.pct_change(1).fillna(0).values
                add_test_portfo(test_portfolios, 'B&H', bh_percentage)
            indexes = calc_return(bh_percentage, test_portfolios)
            results2.append(indexes)

        results2_df = pd.concat(results2, axis=1)
        results2_bh = results2_df["B&H"]
        del results2_df["B&H"]
        final = pd.concat([
            results2_bh,
            results2_df.median(axis=1)
        ], axis=1)
        final.columns = [f"{_file}-B&H", f"{_file}-{arm['name']}-{arm['lamb']}"]
        if idx > 0:
            del final[f"{_file}-B&H"]
        results.append(final)


AAPL


100%|██████████| 2/2 [00:01<00:00,  1.31it/s]


In [24]:
results_df = pd.concat(results, axis=1)
# cols = results_df.columns
# ls = []
# for col in cols:
#     if "CVX" in col:
#         ls.append(col)

results_df

,GS-B&H,GS-profit-0,GS-regularized-0.01,GS-regularized-0.05,GS-regularized-0.1,GS-regularized-0.2,GS-sharpe-0.01,GS-volatility-10
sortino_test,0.716385,3.477152,4.534565,0.0,0.0,0.0,0.639747,0.424042
sharpe_test,0.522315,2.360674,2.736183,0.0,0.0,0.0,0.468970,0.303545
risk_test,0.338601,0.273724,0.232215,0.0,0.0,0.0,0.336912,0.179161
mdd_test,0.487488,0.330980,0.258174,0.0,0.0,0.0,0.498524,0.456210
downrisk_test,0.246873,0.186250,0.140726,0.0,0.0,0.0,0.246848,0.174057
cumreturn_test,0.613265,10.585472,10.144479,0.0,0.0,0.0,0.501814,0.171231


In [7]:
# results_df.to_csv("singl_reward.csv")

,AAPL-B&H,AAPL-profit-0,AAPL-regularized-0.01,AAPL-regularized-0.05,AAPL-regularized-0.1,AAPL-regularized-0.2,AAPL-sharpe-0.01,AAPL-volatility-10,AMGN-B&H,AMGN-profit-0,...,DIS-sharpe-0.01,DIS-volatility-10,GS-B&H,GS-profit-0,GS-regularized-0.01,GS-regularized-0.05,GS-regularized-0.1,GS-regularized-0.2,GS-sharpe-0.01,GS-volatility-10
sortino_test,1.722050,2.591283,2.758458,3.268644,2.244711,0.0,1.810705,1.515620,0.660905,2.949069,...,0.660723,0.190089,0.716385,3.477152,4.534565,0.0,0.0,0.0,0.639747,0.424042
sharpe_test,1.277563,1.871630,1.983001,2.180695,1.341795,0.0,1.320487,1.099999,0.466487,1.920073,...,0.459625,0.134109,0.522315,2.360674,2.736183,0.0,0.0,0.0,0.468970,0.303545
risk_test,0.328437,0.300166,0.293674,0.248992,0.160871,0.0,0.317817,0.222016,0.267469,0.220392,...,0.312989,0.245921,0.338601,0.273724,0.232215,0.0,0.0,0.0,0.336912,0.179161
mdd_test,0.385159,0.314273,0.314273,0.296056,0.178112,0.0,0.365095,0.334128,0.245339,0.216721,...,0.438324,0.438324,0.487488,0.330980,0.258174,0.0,0.0,0.0,0.498524,0.456210
downrisk_test,0.243663,0.215809,0.211147,0.165429,0.097144,0.0,0.233757,0.165422,0.188788,0.142606,...,0.217739,0.185008,0.246873,0.186250,0.140726,0.0,0.0,0.0,0.246848,0.174057
cumreturn_test,3.311191,6.953515,7.711565,6.478080,1.359441,0.0,3.392311,1.448604,0.428277,4.053379,...,0.466293,0.022033,0.613265,10.585472,10.144479,0.0,0.0,0.0,0.501814,0.171231
